## Разметка датасета

## Подготовка

In [ ]:
import numpy as np
import pandas as pd


df = pd.read_csv(
    "vk/vk_messages_mriya_0.csv",
    encoding="utf-8",
    index_col=0,
    dtype={
        0: "int64",
        1: "int64",
        2: "object",
        3: "int8",
        4: "float64",
        5: "object",
        6: "object",
        7: "object",
    },
)
df

,date,text,service_id,rating,answer,additional_id,name
0,1744982287,ТОП‑5 роскошных отелей Южного берега Крыма — н...,5,0.0,NaN,-227189389.0,-227189389
1,1744981734,🌻‍Как изменился туризм в Крыму за 10 лет: ТОП-...,5,0.0,NaN,-123688641.0,-123688641
2,1744902303,ТОП‑5 роскошных отелей Южного берега Крыма на ...,5,0.0,NaN,NaN,447123612
3,1744819881,🏆 ТОП-10 лучших отелей Крыма 2025 года\nРейтин...,5,0.0,NaN,-178540304.0,-178540304
4,1744809009,"Джиптур - это не про цветочное безумие, но куд...",5,0.0,NaN,-75616835.0,-75616835
...,...,...,...,...,...,...,...
519,1704286081,🎄Топ-5 туристических «магнитов» Ялты \n \nИзуч...,5,NaN,NaN,-153002451.0,-153002451
520,1704286081,🎄Топ-5 туристических «магнитов» Ялты \n \nИзуч...,5,NaN,NaN,-153002451.0,-153002451
521,1704286081,🎄Топ-5 туристических «магнитов» Ялты \n \nИзуч...,5,NaN,NaN,-153002451.0,-153002451
522,1704286081,🎄Топ-5 туристических «магнитов» Ялты \n \nИзуч...,5,NaN,NaN,-153002451.0,-153002451


In [19]:
df = df[df.groupby('text')['text'].transform('count') == 1]
df

,date,text,service_id,rating,answer,additional_id,name
0,1744982287,ТОП‑5 роскошных отелей Южного берега Крыма — н...,5,0.0,NaN,-227189389.0,-227189389
1,1744981734,🌻‍Как изменился туризм в Крыму за 10 лет: ТОП-...,5,0.0,NaN,-123688641.0,-123688641
2,1744902303,ТОП‑5 роскошных отелей Южного берега Крыма на ...,5,0.0,NaN,NaN,447123612
3,1744819881,🏆 ТОП-10 лучших отелей Крыма 2025 года\nРейтин...,5,0.0,NaN,-178540304.0,-178540304
4,1744809009,"Джиптур - это не про цветочное безумие, но куд...",5,0.0,NaN,-75616835.0,-75616835
...,...,...,...,...,...,...,...
498,1704961579,После долгих каникул самое время помечтать о с...,5,NaN,NaN,NaN,19104574
499,1704885266,Получи профессию «Матрос-спасатель» - проведи ...,5,NaN,NaN,-219287327.0,-219287327
500,1704641073,На сцене театра имени А.П.Чехова состоялся уже...,5,NaN,NaN,NaN,605091785
501,1704603602,🎡 Парк развлечений Dreamwood.\n\n📍Находится па...,5,NaN,NaN,-188034120.0,-188034120


In [20]:
from difflib import SequenceMatcher
import itertools

COMPARE_COEF = 0.95

def string_compare(s1, s2):
    res = SequenceMatcher(None, s1, s2).ratio()
    return res

texts = df['text'].tolist()

# Словарь для хранения дубликатов
duplicate_groups = {}
visited = set()

# Сравниваем каждую уникальную пару строк
for i, j in itertools.combinations(range(len(texts)), 2):
    if i in visited or j in visited:
        continue  # Пропускаем уже обработанные
    
    s1, s2 = texts[i], texts[j]
    if string_compare(s1, s2) >= COMPARE_COEF:
        # Объединяем группы или создаём новую
        group_i = duplicate_groups.get(i, {i})
        group_j = duplicate_groups.get(j, {j})
        merged_group = group_i.union(group_j)
        for idx in merged_group:
            duplicate_groups[idx] = merged_group
        visited.update(merged_group)

# Находим уникальные строки (не входящие в группы)
unique_indices = [i for i in range(len(texts)) if i not in visited]

# Фильтруем DataFrame
result_df = df.loc[unique_indices]
result_df

KeyboardInterrupt: 

## Разметка

In [4]:
labels = df['rating'].to_list()
for i, line in df.iterrows():
    if labels[i].is_integer() or line['text'] is np.nan:
        continue

    text = line['text']
    print(i)
    if len(text) > 100:
        curlen = 0
        for subtext in text.split(' '):
            print(subtext, end=' ')
            curlen += len(subtext)
            if curlen > 100:
                curlen = 0
                print()
    else:
        print(text)

    print('\n')
    
    flag = False

    try:
        label = None
        while label is None:
            try:
                label = int(input(f"{i} сообщение\nКатегория (0 - neutral, 1 - positive, 2 - negative):\n"))
                if label == -1:
                    label = None
                    flag = True
                    break
            except ValueError:
                label = None
    except:
        break
    
    if flag:
        break

    labels[i] = label
    print()
    
# [labels.append(None) for _ in range(df.shape[0] - len(labels))]

df['rating'] = labels

0
ТОП‑5 роскошных отелей Южного берега Крыма — наш личный выбор на 2025 💎

Мы изучили десятки вариантов, опросили гостей, 
прошлись по территориям и интерьерам — и собрали для вас рейтинг отелей, которые действительно впечатляют. Сервисы, атмосфера, 
архитектура, эмоции — всё это складывается в уникальный опыт. Вот, что мы выбрали 👇

1⃣ Mriya Resort & SPA 5★ 
Отель с 399 
номерами и 37 виллами, который предлагает 24 ресторана и бара, три тематических парка, включая Винный парк WinePark. Здесь 
есть SPA-центр с бассейнами, медицинский центр, спортивные зоны, теннисные корты и фитнес-центр. Сервис на высшем уровне, 
а территория отеля дополнена зелеными зонами и винными садами. Конгресс-центр и детский клуб — отличное место для отдыха 
и бизнес-мероприятий.

2⃣ Luciano Wellness & SPA Foros 5★
Если ваш приоритет — восстановление и гармония, Luciano — для 
вас. Находится в Форосском парке, окруженный хвойным лесом и морем. Этот отель предлагает персонализированные оздоровительные 
программ

62-0

In [5]:
df.loc[~df['rating'].isna(), ['text', 'rating']]

,text,rating
0,ТОП‑5 роскошных отелей Южного берега Крыма — н...,0.0
1,🌻‍Как изменился туризм в Крыму за 10 лет: ТОП-...,0.0
2,ТОП‑5 роскошных отелей Южного берега Крыма на ...,0.0
3,🏆 ТОП-10 лучших отелей Крыма 2025 года\nРейтин...,0.0
4,"Джиптур - это не про цветочное безумие, но куд...",0.0
...,...,...
315,🏆 Представители Свердловской области стали обл...,0.0
316,Премия «Россия — мои горизонты» престижное и в...,0.0
317,Где полюбоваться сакурой в Крыму? А где — тюль...,0.0
318,Крым. Ялта. MRIYA RESORT \n\n#mriyaresort #mri...,0.0


In [ ]:
df.loc['rating'].info()

KeyError: 'rating'

In [6]:
df.dropna(axis=0, how='all').to_csv("vk/vk_messages_mriya_0.csv", index=False)

## Предподготовка данных

In [1]:
import numpy as np
import pandas as pd


max_samples = 60000  # максимальное количество элементов в датасете

def sample_dataset(df, sample_size):
    proportions = df['label'].value_counts(normalize=True)
    return (df.groupby('label')
            .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))
            .reset_index(drop=True))


test_datasets = {}
df = pd.read_csv("/kaggle/input/russian-sentiment-analysis-test-dataset/tg_messages_mriya.csv")
test_datasets['tg_messages_mriya'] = df[['text', 'rating']].rename(columns={'rating': 'label'})
test_datasets['tg_messages_mriya'].head()

,text,label
0,Ну Мрия это ещё не весь Крым☺️,0
1,МРИЯ - это КОСМОС❤️,1
2,"__🧑‍🚀 Поехали!__\n\nНе только в космос, но и в...",1
3,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,1
4,**Мечты о звёздах начинаются с Земли!**\n\nИ е...,1


### Sentiment of bank reviews

In [2]:
df = pd.read_csv("/kaggle/input/russian-sentiment-analysis-test-dataset/sentiment_of_bank_reviews.csv", sep='\t')
df.head()

,idx,Score,Text
0,0,Positive,В Альфа-Банке работает замечательная девушка -...
1,1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,2,Positive,Очень порадовала оперативность работы в банке....
3,3,Negative,Имела неосторожность оформить потреб. кредит в...
4,4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...


In [3]:
df['Score'].value_counts()

Score
Positive    7000
Negative    6999
Name: count, dtype: int64

In [4]:
df = df[['Text', 'Score']].rename(columns={'Text': 'text', 'Score': 'label'})
label_repl = {'Negative': 2, 'Positive': 1}
df['label'] = df['label'].apply(lambda x: label_repl[x])

test_datasets['sentiment_of_bank_reviews'] = df
test_datasets['sentiment_of_bank_reviews'].head()

,text,label
0,В Альфа-Банке работает замечательная девушка -...,1
1,Оформляя рассрочку в м. Видео в меге тёплый ст...,2
2,Очень порадовала оперативность работы в банке....,1
3,Имела неосторожность оформить потреб. кредит в...,2
4,Небольшая предыстория: Нашел на сайте MDM банк...,2


### RuTweetCorp

In [5]:
df_pos = pd.read_csv('/kaggle/input/russian-sentiment-analysis-test-dataset/RuTweetCorp_positive.csv')
df_neg = pd.read_csv('/kaggle/input/russian-sentiment-analysis-test-dataset/RuTweetCorp_negative.csv')
df_pos.head()

,id,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [6]:
df_pos = pd.concat([df_pos[['ttext']], pd.DataFrame({'label': [1] * len(df_pos)})], axis=1)
df_neg = pd.concat([df_neg[['ttext']], pd.DataFrame({'label': [2] * len(df_neg)})], axis=1)
df = pd.concat([df_pos, df_neg], axis=0).rename(columns={'ttext': 'text'})
df = df.sample(frac=1).reset_index(drop=True)

test_datasets['rutweetcorp'] = df
test_datasets['rutweetcorp'].head()

,text,label
0,в фильме:\n- я вопрос спросил... \n- я ответ о...,1
1,"Будет ли сегодня ""день наоборот"" или нет, скор...",1
2,"RT @bulomisesab: Блин, GVIM, будучи развёрнуты...",2
3,выпускникам удачи завтра на русском) мы сможем...,1
4,RT @KarineAzazaza: @Angel___Anya200 ага и в ка...,1


In [7]:
test_datasets['rutweetcorp']['label'].value_counts()

label
1    114911
2    111923
Name: count, dtype: int64

In [8]:
test_datasets['rutweetcorp'] = sample_dataset(test_datasets['rutweetcorp'], max_samples)
test_datasets['rutweetcorp']['label'].value_counts()

/tmp/ipykernel_31/2383192684.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))


label
1    30395
2    29605
Name: count, dtype: int64

### RuReviews

In [9]:
df = pd.read_csv('/kaggle/input/russian-sentiment-analysis-test-dataset/rureviews.csv', sep='\t')
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [10]:
df['sentiment'].value_counts()

sentiment
negative    30000
neautral    30000
positive    30000
Name: count, dtype: int64

In [11]:
df = df.rename(columns={'review': 'text', 'sentiment': 'label'})
label_repl = {'negative': 2, 'neautral': 0, 'positive': 1}
df['label'] = df['label'].apply(lambda x: label_repl[x])

test_datasets['rureviews'] = df
test_datasets['rureviews'].head()

,text,label
0,качество плохое пошив ужасный (горловина напер...,2
1,"Товар отдали другому человеку, я не получила п...",2
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",2
3,"товар не пришел, продавец продлил защиту без м...",2
4,"Кофточка голая синтетика, носить не возможно.",2


In [12]:
test_datasets['rureviews'] = sample_dataset(test_datasets['rureviews'], max_samples)
test_datasets['rureviews']['label'].value_counts()

/tmp/ipykernel_31/2383192684.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))


label
0    20000
1    20000
2    20000
Name: count, dtype: int64

### RuSentiment

In [13]:
df_preselected = pd.read_csv('/kaggle/input/russian-sentiment-analysis-test-dataset/rusentiment_preselected_posts.csv')
df_random = pd.read_csv('/kaggle/input/russian-sentiment-analysis-test-dataset/rusentiment_random_posts.csv')
df_test = pd.read_csv('/kaggle/input/russian-sentiment-analysis-test-dataset/rusentiment_test.csv')
df = pd.concat([df_preselected, df_random, df_test], axis=0)[['text', 'label']]
df.head()

,text,label
0,Прорвём информационную блокаду изнутри.,neutral
1,"Никогда у меня не будет ""одного приложения для...",negative
2,"Кури-и тебя не укусит злая собака, потому что ...",skip
3,"Есть 3 типа людей:\nУмные, которые делают все ...",neutral
4,мегафон чет накрыло,neutral


In [14]:
df['label'].value_counts()

label
neutral     12720
positive     6646
skip         4440
negative     3912
speech       3467
Name: count, dtype: int64

In [15]:
label_repl = {'neutral': 0, 'positive': 1, 'skip': 0, 'negative': 2, 'speech': 0}
df['label'] = df['label'].apply(lambda x: label_repl[x])

test_datasets['rusentiment'] = df
test_datasets['rusentiment'] .head()

,text,label
0,Прорвём информационную блокаду изнутри.,0
1,"Никогда у меня не будет ""одного приложения для...",2
2,"Кури-и тебя не укусит злая собака, потому что ...",0
3,"Есть 3 типа людей:\nУмные, которые делают все ...",0
4,мегафон чет накрыло,0


### ru_sentiment_MonoHime

In [16]:
df = pd.read_csv('/kaggle/input/russian-sentiment-analysis-test-dataset/ru_sentiment_monohime.csv', index_col=0)
df.head()

,text,sentiment
43956,Развода на деньги нет\nНаблюдаюсь в Лайфклиник...,1
17755,Отель выбрали потому что рядом со стадионом. О...,0
20269,"Вылечили\nГноился с рождения глазик, в поликли...",1
16648,Хорошее расположение.С вокзала дошли пешком.Но...,0
27879,"Отличное месторасположение,прекрасный вид,особ...",1


In [17]:
df['sentiment'].value_counts()

sentiment
1    100792
2     55310
0     54887
Name: count, dtype: int64

In [18]:
df = df.rename(columns={'sentiment': 'label'}).reset_index(drop=True)
test_datasets['ru_sentiment_monohime'] = df
test_datasets['ru_sentiment_monohime'].head()

,text,label
0,Развода на деньги нет\nНаблюдаюсь в Лайфклиник...,1
1,Отель выбрали потому что рядом со стадионом. О...,0
2,"Вылечили\nГноился с рождения глазик, в поликли...",1
3,Хорошее расположение.С вокзала дошли пешком.Но...,0
4,"Отличное месторасположение,прекрасный вид,особ...",1


In [19]:
test_datasets['ru_sentiment_monohime'] = sample_dataset(test_datasets['ru_sentiment_monohime'], max_samples)
test_datasets['ru_sentiment_monohime']['label'].value_counts()

/tmp/ipykernel_31/2383192684.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(round(sample_size*proportions[x.name]), random_state=42))


label
1    28663
2    15729
0    15608
Name: count, dtype: int64

In [20]:
test_datasets

{'tg_messages_mriya':                                                   text  label
 0                       Ну Мрия это ещё не весь Крым☺️      0
 1                                  МРИЯ - это КОСМОС❤️      1
 2    __🧑‍🚀 Поехали!__\n\nНе только в космос, но и в...      1
 3    **Мечты о звёздах начинаются с Земли!**\n\nИ е...      1
 4    **Мечты о звёздах начинаются с Земли!**\n\nИ е...      1
 ..                                                 ...    ...
 329  Мы уверены, что форум [Новые горизонты](https:...      0
 330  Мы уверены, что форум [Новые горизонты](https:...      0
 331  Фрагмент выступления генерального директора ку...      0
 332  Фрагмент выступления генерального директора ку...      0
 333  __В форуме __[__Новые горизонты__](https://t.m...      0
 
 [334 rows x 2 columns],
 'sentiment_of_bank_reviews':                                                     text  label
 0      В Альфа-Банке работает замечательная девушка -...      1
 1      Оформляя рассрочку в м. Видео

## Тестирование

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix
)

MODEL_NAMES = [
    # "blanchefort/rubert-base-cased-sentiment",
    # "seara/rubert-tiny2-russian-sentiment",
    # "cointegrated/rubert-tiny-sentiment-balanced",
    # "MonoHime/rubert_conversational_cased_sentiment",
    # "sismetanin/rubert-ru-sentiment-rureviews",
    # "blanchefort/rubert-base-cased-sentiment-rurewiews",
    # "rajora/distilbert-multilingual-sentiment"
    # "sismetanin/mbart_ru_sum_gazeta-ru-sentiment-rusentiment",
    "sismetanin/mbart_ru_sum_gazeta-ru-sentiment-rureviews",
    "kartashoffv/vashkontrol-sentiment-rubert",
    "tabularisai/multilingual-sentiment-analysis"
]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 12
MAX_LENGTH = 512

In [22]:
import gc
from tqdm import tqdm


class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

def evaluate_model(model, dataloader, model_name, label_consistance=True):
    model.eval()
    predictions, true_labels = [], []

    i = 1
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Evaluating {model_name}"):
            try:
                inputs = {
                    "input_ids": batch["input_ids"].to(DEVICE),
                    "attention_mask": batch["attention_mask"].to(DEVICE)
                }
                outputs = model(**inputs)
                logits = outputs.logits
                if not label_consistance:
                    if model.config.num_labels == 3:
                        logits[:, 1] = torch.max(logits[:, :2], dim=1)[0]
                        logits = logits[:, 1:]
                    elif model.config.num_labels == 5:
                        logits[:, 1] = torch.max(logits[:, 1:], dim=1)[0]
                        logits = torch.column_stack([logits[:, 1], logits[:, 0]])
                
                predictions.extend(torch.argmax(logits, dim=1).cpu().tolist())
                true_labels.extend(batch["labels"].cpu().tolist())

                if i % 25 == 0:
                    del inputs
                    del outputs
                    gc.collect()

                i += 1
            finally:
                continue

    return predictions, true_labels

In [ ]:
import time

results = []
for model_name in MODEL_NAMES:
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name).to(DEVICE)
    except Exception as e:
        print(f"Ошибка загрузки модели {model_name}: {str(e)}")
        raise e

    time.sleep(30)
    for dataset_name, df in list(test_datasets.items())[3:]:
        try:
            texts = df["text"].tolist()
            labels = df["label"].tolist()

            labels_are_consistent = model.config.num_labels == 2
            labels = [label if label == 0 else label - 1 for label in labels]

            dataset = CustomDataset(texts, labels, tokenizer)
            dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, pin_memory=True)

            preds, truth = evaluate_model(model, dataloader, model_name, labels_are_consistent)
        
            metrics = {
                "model": model_name,
                "dataset": dataset_name,
                "accuracy": accuracy_score(truth, preds),
                "f1": f1_score(truth, preds, zero_division=0.),
                "f1_weighted": f1_score(truth, preds, average="weighted", zero_division=0.),
                "f1_macro": f1_score(truth, preds, average="macro", zero_division=0.),
                "classification_report": classification_report(truth, preds, zero_division=0.),
                "confusion_matrix": confusion_matrix(truth, preds).tolist()
            }
            print(metrics)
            results.append(metrics)
        finally:
            results_df = pd.DataFrame(results)
            results_df.to_csv("/kaggle/working/model_evaluation_results5.csv", index=False)
            results_df.to_excel("/kaggle/working/model_evaluation_results5.xlsx", index=False)


tokenizer_config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

2025-04-21 10:02:21.338338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745229741.748511      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745229741.883447      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/2.45G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.45G [00:00<?, ?B/s]

Evaluating sismetanin/mbart_ru_sum_gazeta-ru-sentiment-rureviews:  45%|████▍     | 2227/5000 [56:30<1:12:09,  1.56s/it]